In [2]:
import pandas as pd
import numpy as np
import sklearn

In [6]:
import nsepy
import datetime
from datetime import date

In [8]:
# Infosys stock
infy = nsepy.get_history(symbol='INFY',
                    start=date(2015, 1, 1), 
                    end=date(2018, 10, 25))

In [9]:
infy.shape

(945, 14)

In [10]:
infy.head(10)

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2015-01-01,INFY,EQ,1972.55,1968.95,1982.00,1956.90,1971.00,1974.40,1971.34,500691,9.870306e+13,14908,258080,0.5154
2015-01-02,INFY,EQ,1974.40,1972.00,2019.05,1972.00,2017.95,2013.20,2003.25,1694580,3.394669e+14,54166,1249104,0.7371
2015-01-05,INFY,EQ,2013.20,2009.90,2030.00,1977.50,1996.00,1995.90,2004.59,2484256,4.979911e+14,82694,1830962,0.7370
2015-01-06,INFY,EQ,1995.90,1980.00,1985.00,1934.10,1965.10,1954.20,1954.82,2416829,4.724458e+14,108209,1772070,0.7332
2015-01-07,INFY,EQ,1954.20,1965.00,1974.75,1950.00,1966.05,1963.55,1962.59,1812479,3.557162e+14,62463,1317720,0.7270
2015-01-08,INFY,EQ,1963.55,1985.60,1997.00,1950.00,1979.25,1973.45,1972.78,3391230,6.690160e+14,92752,2686012,0.7920
2015-01-09,INFY,EQ,1973.45,1980.10,2109.00,1913.05,2075.30,2074.45,2037.69,11215832,2.285439e+15,359214,3369489,0.3004
2015-01-12,INFY,EQ,2074.45,2092.00,2119.20,2075.00,2112.95,2115.95,2099.40,3189722,6.696516e+14,107209,1818800,0.5702
2015-01-13,INFY,EQ,2115.95,2107.80,2107.80,2075.00,2092.00,2088.90,2089.42,2200309,4.597374e+14,66676,1385009,0.6295


In [14]:
dates = infy.index.values

day = [date.day for date in dates]
mon = [date.month for date in dates]
infy_prev_close = infy['Prev Close'].values.reshape(1,-1)
stock_price = infy.Close.values

In [15]:
X = np.dstack((day, mon, infy_prev_close))[0]
y = stock_price

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [18]:
lin = LinearRegression()
ran = RandomForestRegressor()

In [19]:
lin.fit(X,y)
ran.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [22]:
ran_nxt = ran.predict([[X[-1][0]+1, X[-1][1], X[-1][2]]])[0]
lm_nxt = lin.predict([[X[-1][0]+1, X[-1][1], X[-1][2]]])[0]
print('Ran Predicted Closing Price for next day : '+str(ran_nxt))
print('Linear Regression predicted closing price for next day : '+str(lm_nxt))

Ran Predicted Closing Price for next day : 652.86
Linear Regression predicted closing price for next day : 653.1808646441414


Prediction for next 20 days

In [25]:

import datetime
ran_pred_prev_close = lm_pred_prev_close = X[-1][2]
day = X[-1][0]
month = X[-1][1]
year = 2018
for i in range(1, 21): 
    d = date(year, int(month), int(day))
    d += datetime.timedelta(days=i) 
    ran_next_day_closing_price = ran.predict([[d.day,d.month, ran_pred_prev_close]])[0]
    ran_pred_prev_close = ran_next_day_closing_price
    lm_next_day_closing_price = lin.predict([[d.day, d.month, lm_pred_prev_close]])[0]
    lm_pred_prev_close = lm_next_day_closing_price
    print('SVM Model predicted closing price for {}-{}-{} as: {:.2f}'.format(d.day, d.month, d.year, ran_next_day_closing_price))
    print('LRM Model predicted closing price for {}-{}-{} as: {:.2f}'.format(d.day, d.month, d.year, lm_next_day_closing_price))
    print()

SVM Model predicted closing price for 26-10-2018 as: 652.86
LRM Model predicted closing price for 26-10-2018 as: 653.18

SVM Model predicted closing price for 27-10-2018 as: 652.86
LRM Model predicted closing price for 27-10-2018 as: 656.69

SVM Model predicted closing price for 28-10-2018 as: 652.86
LRM Model predicted closing price for 28-10-2018 as: 660.33

SVM Model predicted closing price for 29-10-2018 as: 652.86
LRM Model predicted closing price for 29-10-2018 as: 664.09

SVM Model predicted closing price for 30-10-2018 as: 652.86
LRM Model predicted closing price for 30-10-2018 as: 667.98

SVM Model predicted closing price for 31-10-2018 as: 652.86
LRM Model predicted closing price for 31-10-2018 as: 671.99

SVM Model predicted closing price for 1-11-2018 as: 670.36
LRM Model predicted closing price for 1-11-2018 as: 670.20

SVM Model predicted closing price for 2-11-2018 as: 674.15
LRM Model predicted closing price for 2-11-2018 as: 668.61

SVM Model predicted closing price fo